In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
from json import load
import tifffile
from tqdm import tqdm
from skimage.draw import polygon
from matplotlib import rcParams
from glasbey import create_palette
import napari
from scipy.spatial.distance import cdist
from sklearn.decomposition import PCA

palette = {
    "green": "#558150",
    "beige": "#F1E2C3",
    "brown": "#A7785A",
    "pink": "#F0D6C2",
    "black": "#0E0E0E",
}

rcParams['font.family'] = 'sans-serif'
rcParams['figure.facecolor'] = "#FFFFFF00"
rcParams['axes.facecolor'] = "#FFFFFF00"
rcParams['legend.framealpha'] = 0.2
rcParams['axes.edgecolor'] = palette["black"]
rcParams['axes.labelcolor'] = palette["black"]
rcParams['xtick.color'] = palette["black"]
rcParams['ytick.color'] = palette["black"]
rcParams['text.color'] = palette["black"]
rcParams['axes.titlecolor'] = palette["black"]

s_palette = sns.cubehelix_palette(as_cmap=True)
pal = sns.color_palette("dark")
cpal = sns.cubehelix_palette(start=-0.25, rot=2, as_cmap=True)
g_palette = create_palette(2000)

save_path = Path(r"D:\OneDrive - Princeton University\Lab Meetings\2025_04_14 Stas\figures")

In [3]:
raw = tifffile.imread(r"D:\Tracking\NucleiTracking\data\interim\lightsheet\2025_03_21_trk\recon\recon_fused_tp_290_ch_0.tif")

from skimage.filters import difference_of_gaussians
from skimage.feature import peak_local_max
from skimage.morphology import ball

dog = difference_of_gaussians(raw, 2, 6)
peaks = peak_local_max(dog, min_distance=5, threshold_abs=35)



  0%|          | 0/5942 [00:00<?, ?it/s]


ValueError: too many values to unpack (expected 2)

In [4]:
peaks

array([[255, 516, 296],
       [270, 370, 274],
       [254, 329, 282],
       ...,
       [168, 439, 337],
       [ 78, 236, 233],
       [269, 610, 248]], dtype=int64)

In [32]:
g_palette = create_palette(2000, as_hex=False)


k = 5

footprint = ball(k)
labels = np.zeros(raw.shape, dtype=int)

for i, peak in tqdm(enumerate(peaks)):
    try: 
        labels[peak[0] - k: peak[0] + k+1, peak[1] - k: peak[1] + k+1, peak[2] - k: peak[2] + k+1][footprint > 0] = i
    except:
        continue
    
colored = np.array(g_palette)[labels % 2000]
colored[labels == 0] = 0



5942it [00:00, 38575.69it/s]


In [33]:
raw2 = raw * (raw > np.median(raw)*2) 

In [39]:
raw3ch = np.stack([raw]*3, axis=-1)
ci = colored * raw3ch
ci[labels == 0] = raw3ch[labels == 0] / 4

In [40]:
viewer = napari.Viewer(ndisplay=3)
viewer.add_image(ci, channel_axis=-1, colormap=['red', 'green', 'blue'])
viewer.add_image(raw)
# img = viewer.add_image(raw3ch, channel_axis=-1)
napari.run()